In [1]:
import pandas
import pandas as pd

from GA import GA
import optproblems.cec2005 as optproblems

# Test Functions
- F2: Shifted Schwefel’s Problem 1.2
    - Uni-modal without noise
- F4: Shifted Schwefel’s Problem 1.2 with Noise in Fitness
    - Uni-modal with noise
- F8: Shifted Rotated Ackley’s Function with Global Optimum on Bounds
    - Single function
    - Global optimum on bounds
    - Multi-modal
- F13: Shifted Expanded Griewank’s plus Rosenbrock’s Function (F8F2)
    - Expanded Function
    - Multi-modal
- F17: Rotated Version of Hybrid Composition Function with Noise in Fitness
    - Gaussian noise in fitness
    - Multi-modal
    - Huge number of local optima
    - In-deterministic
- F24: Rotated Hybrid Composition Function
    - Composite function
    - Huge number of local optima
    - Multi-modal

Hyper parameters

In [2]:
# F2 :   Unimodal without noise
# F4 :   Unimodal with noise
# F8 :   Single Function, Global Optimum on Bounds, Multi-modal
# F13:   Expanded Function, Multi-modal
# F17:   Gaussian noise in Fitness, Multi-modal, Huge number of local optima, indeterministic
# F24:   Composite Function, Huge number of local optima, Multi-modal

run_configuration_suite = False
if run_configuration_suite:
    benchmarks_and_bounds = [optproblems.F2, [-100,100], "F2", optproblems.F4, [-100,100], "F4", optproblems.F8, [-32,32], "F8", optproblems.F13, [-3,1], "F13", optproblems.F17, [-5,5], "F17", optproblems.F24, [-5,5], "F24"]
    no_dimensions = [10, 30, 50]
    pop_sizes = [5, 10, 15]
    num_generations = [50, 150]
    tournament_t = [2, 4]
    crossover_percentage = [0.5, 0.95, 1]
    mutation_rates = [0.2, 0.8, 1]
    decreasing_mutation_rate = [True, False]
    elites = [0, 1]

    average_over_range = 5

    list_for_df = []
    for i in range(0, len(benchmarks_and_bounds), 3):
        number_of_functions = (len(benchmarks_and_bounds)/3)
        print(f"Progress: {i/3} / {number_of_functions}")
        bound = benchmarks_and_bounds[i+1]
        benchmark_name = benchmarks_and_bounds[i+2]
        for no_dimension in no_dimensions:
            print(f"\tDimension: {no_dimension}")
            benchmark = benchmarks_and_bounds[i](no_dimension)
            optimal_solution = benchmark.get_optimal_solutions()
            benchmark.evaluate(optimal_solution[0])
            optimal_minimum = optimal_solution[0].objective_values
            for pop_size in pop_sizes:
                print(f"\t\tPopulation: {pop_size}")
                for t in tournament_t:
                    for crossover in crossover_percentage:
                        for mutation in mutation_rates:
                            for dmr in decreasing_mutation_rate:
                                for elite in elites:
                                    for generations in num_generations:
                                        average_fitness = 0
                                        average_difference_from_optimal = 0

                                        best_fitness = 0
                                        best_time = 0
                                        best_difference_from_optimal = 0

                                        for j in range(average_over_range):
                                            algorithm = GA(benchmark, no_dimension, bound, pop_size, num_generations=generations, t=t, crossover_percentage=crossover, mutation_rate=mutation, decreasing_mutation_rate=dmr, elite=elite)
                                            result, time = algorithm.run()
                                            fitness = result.fitness
                                            diff_from_optimal = fitness - optimal_minimum

                                            average_fitness += 1/average_over_range * fitness
                                            average_difference_from_optimal += 1/average_over_range * diff_from_optimal

                                            if j == 0:
                                                best_fitness = fitness
                                                best_difference_from_optimal = diff_from_optimal
                                                best_time = time
                                                continue

                                            if fitness < best_fitness:
                                                best_fitness = fitness

                                            if diff_from_optimal < best_difference_from_optimal:
                                                best_difference_from_optimal = diff_from_optimal

                                            if time < best_time:
                                                best_time = time

                                        result_and_configuration = [benchmark_name, optimal_minimum, best_fitness, best_difference_from_optimal, average_fitness, average_difference_from_optimal, best_time, generations, no_dimension, pop_size, t, crossover, mutation, dmr, elite, bound, average_over_range]
                                        list_for_df.append(result_and_configuration)

    labels = ["TestFunction", "OptimalMinimum", "BestFitness", "BestDiffFromOpt", "AverageFitness", "AverageDiffFromOpt", "TimePerRun", "Generations", "NoDimension", "PopSize", "TForTournament", "CrossoverPerc", "MutationRate", "DecreasingMutationRate", "Elite", "Bounds", "AverageOverRange"]
    df = pandas.DataFrame(data=list_for_df, columns=labels)
    df.to_pickle("GA_df.pkl")
else:
    df = pd.read_pickle("GA_df.pkl")

In [3]:
df

,TestFunction,OptimalMinimum,BestFitness,BestDiffFromOpt,AverageFitness,AverageDiffFromOpt,TimePerRun,Generations,NoDimension,PopSize,TForTournament,CrossoverPerc,MutationRate,DecreasingMutationRate,Elite,Bounds,AverageOverRange
0,F2,-450.0,17690.525955,18140.525955,32133.289645,32583.289645,0.011128,50,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
1,F2,-450.0,14235.020676,14685.020676,22384.591201,22834.591201,0.027806,150,10,5,2,0.5,0.2,True,0,"[-100, 100]",5
2,F2,-450.0,6013.072511,6463.072511,14872.551135,15322.551135,0.010348,50,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
3,F2,-450.0,1868.742268,2318.742268,5054.818636,5504.818636,0.031108,150,10,5,2,0.5,0.2,True,1,"[-100, 100]",5
4,F2,-450.0,21611.965380,22061.965380,51345.862343,51795.862343,0.009959,50,10,5,2,0.5,0.2,False,0,"[-100, 100]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,F24,260.0,1604.662489,1344.662489,1633.768568,1373.768568,3.663990,150,50,15,4,1.0,1.0,True,1,"[-5, 5]",5
1292,F24,260.0,2013.695892,1753.695892,2032.972450,1772.972450,1.252370,50,50,15,4,1.0,1.0,False,0,"[-5, 5]",5
1293,F24,260.0,2022.287128,1762.287128,2078.779992,1818.779992,3.872165,150,50,15,4,1.0,1.0,False,0,"[-5, 5]",5
1294,F24,260.0,1880.468349,1620.468349,1907.107345,1647.107345,1.220936,50,50,15,4,1.0,1.0,False,1,"[-5, 5]",5
